In [1]:
import pandas as pd
import torch
import os

from torch.utils.data import Dataset
from transformers import (BertTokenizer, BertForSequenceClassification, Trainer,
                          TrainingArguments, BertPreTrainedModel)
# from simpletransformers.language_modeling import LanguageModelingModel
from sklearn.metrics import accuracy_score, f1_score


In [2]:
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))
print(torch.cuda.get_device_properties(0))

True
NVIDIA GeForce RTX 3060
_CudaDeviceProperties(name='NVIDIA GeForce RTX 3060', major=8, minor=6, total_memory=12050MB, multi_processor_count=28)


In [3]:
MODEL_NAME = '/home/abdullah/Code/dl/lt_bert/best_models/pruned_best_tiny_bert'
TRAIN_FILE_LOC = '/home/abdullah/Code/dl/bnlp-resources/sentiment/merged_train.csv'
EVAL_FILE_LOC = '/home/abdullah/Code/dl/bnlp-resources/sentiment/merged_test.csv'


In [4]:
def tsv_to_text(tsv_file_loc):
    file_name = tsv_file_loc.split("/")[-1].split(".")[0]
    txt_name = tsv_file_loc.replace(".tsv", ".txt")
    txt_name = txt_name.replace("split_merged", "texts")

    if os.path.exists(txt_name):
        return [txt_name, file_name]

    df = pd.read_csv(tsv_file_loc, sep="\t")

    for txt in df["text"]:
        with open(txt_name, "a", encoding="utf8") as f:
            f.writelines(txt + "\n")
    return [txt_name, file_name]


tsv_to_text(TRAIN_FILE_LOC)


['/home/abdullah/Code/dl/bnlp-resources/sentiment/merged_train.csv',
 'merged_train']

In [5]:
def csv_to_df(csv_file_loc):
    df = pd.read_csv(csv_file_loc)
    # remove id column
    # df = df.drop(columns=['id'])

    # replace neutral with 0, positive with 1, negative with 2 in class_label column
    df.loc[df['label'] == 'Neutral', 'label'] = 0
    df.loc[df['label'] == 'Positive', 'label'] = 1
    df.loc[df['label'] == 'Negative', 'label'] = 2

    return df


train_df = csv_to_df(TRAIN_FILE_LOC)

texts = train_df['text'].tolist()
print(train_df)


                                                    text label
0         @BoyBanglar ভালোই,, তাও ত বাংলার বেকার ছেলে না     1
1      'সুন্নাহের গুরুত্ব ও প্রয়োজনীয়তাhttp:/t.co/Kv9...     0
2      সারা দুনিয়া থেকে যদি বাছাই করে ৫০০ জন ভালো মা...     0
3      ক্রিকেট বোর্ড কে যদি দুদকের সম্মুখীন করা হয় এব...     2
4      সপ্তাহের জন্য তারা একটি দুর্দান্ত কাজ করে এই র...     1
...                                                  ...   ...
18739  RT @arafatul: গত এক দশক ধরে জার্মান রাজনীতি খু...     0
18740  না অাছে কোনো পাওয়ার হিটার, না অাছে একটা লেগস্প...     2
18741  গাজীপুরের শ্রীপুরে ১০০ পিছ ইয়াবা ও গাঁজাসহ ৩ জ...     2
18742   করো উন্নতি হউক বা না হউক মেডামের শরীরের ভালো ...     2
18743  খেলোয়াড়দের কথা বাদ দিলাম। বোর্ড, কোচ, নির্বাচক...     2

[18744 rows x 2 columns]


In [6]:
def compute_metrics(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  # calculate accuracy using sklearn's function
  acc = accuracy_score(labels, preds)
  return {
      'accuracy': acc,
  }


In [7]:
def f1_calculator(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  # calculate accuracy using sklearn's function
  f1 = f1_score(labels, preds, average='weighted')
  return {
      'f1': f1,
  }


In [8]:
train_list = tsv_to_text(TRAIN_FILE_LOC)
eval_list = tsv_to_text(EVAL_FILE_LOC)


In [9]:
os.environ["WANDB_DISABLED"] = "true"

train_df = csv_to_df(TRAIN_FILE_LOC)
eval_df = csv_to_df(EVAL_FILE_LOC)


class TINYDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(value[idx])
                for key, value in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)


tokenizer = BertTokenizer.from_pretrained(MODEL_NAME)

train_encodings = tokenizer(train_df['text'].tolist(), padding=True, truncation=True, max_length=512)
eval_encodings = tokenizer(eval_df['text'].tolist(), padding=True, truncation=True, max_length=512)

train_dataset = TINYDataset(train_encodings, train_df['label'].tolist())
eval_dataset = TINYDataset(eval_encodings, eval_df['label'].tolist())

model = BertForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=3).to('cuda')
model.manual_seed = 14


Some weights of the model checkpoint at /home/abdullah/Code/dl/lt_bert/best_models/pruned_best_tiny_bert were not used when initializing BertForSequenceClassification: ['bert.encoder.layer.1.output.dense.weight_mask', 'bert.encoder.layer.0.attention.self.query.weight_mask', 'bert.encoder.layer.1.attention.output.dense.weight_mask', 'cls.predictions.transform.LayerNorm.bias', 'bert.encoder.layer.0.output.dense.weight_orig', 'bert.encoder.layer.0.attention.self.query.weight_orig', 'bert.encoder.layer.1.intermediate.dense.weight_mask', 'bert.encoder.layer.0.attention.output.dense.weight_orig', 'bert.encoder.layer.1.output.dense.weight_orig', 'bert.encoder.layer.0.attention.output.dense.weight_mask', 'bert.encoder.layer.1.attention.self.query.weight_mask', 'cls.predictions.transform.LayerNorm.weight', 'bert.encoder.layer.0.attention.self.value.weight_orig', 'bert.encoder.layer.1.intermediate.dense.weight_orig', 'bert.encoder.layer.0.intermediate.dense.weight_orig', 'bert.encoder.layer.1.at

## 1 Epoch Train

In [10]:
training_args = TrainingArguments(
    output_dir=f"temp",
    num_train_epochs=1,
    per_device_train_batch_size=48,
    per_device_eval_batch_size=96,
    warmup_steps=500,
    learning_rate=5e-5,
    weight_decay=0.01,
    overwrite_output_dir=True,
    logging_dir=f"temp/logs",
    logging_steps=130,
    save_steps=130,
    load_best_model_at_end=True,
    evaluation_strategy="steps",
    seed=14,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset

)

trainer.train()


Using the `WAND_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
***** Running training *****
  Num examples = 18744
  Num Epochs = 1
  Instantaneous batch size per device = 48
  Total train batch size (w. parallel, distributed & accumulation) = 48
  Gradient Accumulation steps = 1
  Total optimization steps = 391
 33%|███▎      | 130/391 [00:07<00:14, 18.62it/s]***** Running Evaluation *****
  Num examples = 4686
  Batch size = 96


{'loss': 1.0859, 'learning_rate': 1.3000000000000001e-05, 'epoch': 0.33}


                                                 
 33%|███▎      | 130/391 [00:08<00:14, 18.62it/s]Saving model checkpoint to temp/checkpoint-130
Configuration saved in temp/checkpoint-130/config.json


{'eval_loss': 1.074790596961975, 'eval_runtime': 1.4909, 'eval_samples_per_second': 3143.163, 'eval_steps_per_second': 32.867, 'epoch': 0.33}


Model weights saved in temp/checkpoint-130/pytorch_model.bin
 66%|██████▋   | 260/391 [00:16<00:06, 19.38it/s]***** Running Evaluation *****
  Num examples = 4686
  Batch size = 96


...

                                                 
100%|█████████▉| 390/391 [00:28<00:00, 19.35it/s]Saving model checkpoint to temp/checkpoint-390
Configuration saved in temp/checkpoint-390/config.json


{'eval_loss': 0.9461308717727661, 'eval_runtime': 1.4958, 'eval_samples_per_second': 3132.682, 'eval_steps_per_second': 32.757, 'epoch': 1.0}


Model weights saved in temp/checkpoint-390/pytorch_model.bin
100%|██████████| 391/391 [00:29<00:00,  2.22it/s]

Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from temp/checkpoint-390 (score: 0.9461308717727661).
100%|██████████| 391/391 [00:29<00:00, 13.18it/s]

{'train_runtime': 29.6652, 'train_samples_per_second': 631.85, 'train_steps_per_second': 13.18, 'train_loss': 1.0494939073577256, 'epoch': 1.0}


TrainOutput(global_step=391, training_loss=1.0494939073577256, metrics={'train_runtime': 29.6652, 'train_samples_per_second': 631.85, 'train_steps_per_second': 13.18, 'train_loss': 1.0494939073577256, 'epoch': 1.0})

## 100 Epoch with 1 epoch model

In [12]:
MODEL_NAME = '/home/abdullah/Code/dl/lt_bert/best_models/1/1_combined_50'
model = BertForSequenceClassification.from_pretrained(
    MODEL_NAME, num_labels=3).to('cuda')
model.manual_seed = 14

for name, param in model.named_parameters():
    if 'bert.encoder.layer.0' in name:
        # print(name, param.requires_grad)
        param.requires_grad = False
# print("\n\n\n")
# for name, param in model.named_parameters():
#     print(name, param.requires_grad)

training_args = TrainingArguments(
    output_dir=f"temp",
    num_train_epochs=100,
    per_device_train_batch_size=192,
    per_device_eval_batch_size=384,
    warmup_steps=500,
    learning_rate=5e-5,
    weight_decay=0.01,
    overwrite_output_dir=True,
    logging_dir=f"temp/logs",
    logging_steps=400,
    save_steps=400,
    load_best_model_at_end=True,
    evaluation_strategy="steps",
    seed=14,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=f1_calculator,

)

trainer.train()

# print(cool.metrics)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,

)

trainer.train()


loading configuration file /home/abdullah/Code/dl/lt_bert/best_models/1/1_combined_50/config.json
Model config BertConfig {
  "_name_or_path": "/home/abdullah/Code/dl/lt_bert/best_models/pruned_best_tiny_bert",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 128,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 512,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 2,
  "num_hidden_layers": 2,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.12.5",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_siz

{'loss': 0.932, 'learning_rate': 4e-05, 'epoch': 4.08}


                                                  
  4%|▍         | 400/9800 [01:13<27:42,  5.66it/s]Saving model checkpoint to temp/checkpoint-400
Configuration saved in temp/checkpoint-400/config.json


{'eval_loss': 0.9268326163291931, 'eval_f1': 0.4224656966846347, 'eval_runtime': 1.7821, 'eval_samples_per_second': 2629.452, 'eval_steps_per_second': 7.295, 'epoch': 4.08}


Model weights saved in temp/checkpoint-400/pytorch_model.bin
  8%|▊         | 800/9800 [02:26<26:48,  5.60it/s]  ***** Running Evaluation *****
  Num examples = 4686
  Batch size = 384


...


 98%|█████████▊| 9600/9800 [29:42<00:36,  5.55it/s]Saving model checkpoint to temp/checkpoint-9600
Configuration saved in temp/checkpoint-9600/config.json


{'eval_loss': 0.9393672347068787, 'eval_accuracy': 0.6384976525821596, 'eval_runtime': 1.7724, 'eval_samples_per_second': 2643.923, 'eval_steps_per_second': 7.335, 'epoch': 97.96}


Model weights saved in temp/checkpoint-9600/pytorch_model.bin
100%|██████████| 9800/9800 [30:19<00:00,  6.16it/s]

Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from temp/checkpoint-800 (score: 0.8244216442108154).
100%|██████████| 9800/9800 [30:19<00:00,  5.39it/s]

{'train_runtime': 1819.5274, 'train_samples_per_second': 1030.158, 'train_steps_per_second': 5.386, 'train_loss': 0.6562452728894292, 'epoch': 100.0}


TrainOutput(global_step=9800, training_loss=0.6562452728894292, metrics={'train_runtime': 1819.5274, 'train_samples_per_second': 1030.158, 'train_steps_per_second': 5.386, 'train_loss': 0.6562452728894292, 'epoch': 100.0})